In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.pydantic_v1 import BaseModel, Field
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END
import os
import json
import time
from dotenv import load_dotenv
from typing import Any, Dict, List, Optional, Literal, TypedDict
from dataclasses import dataclass, asdict

c:\a\envs\env2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", 
    api_key=GOOGLE_API_KEY,
    temperature=0.1)

In [4]:
llm.invoke("Hello, world!").content

'Hello, world!\n'

In [25]:
class Gemini:
    def __init__(self):
        self.model = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            api_key=GOOGLE_API_KEY,
            temperature=0.1
        )

    def complete(self, system: str, user: str, **kwargs) -> str:
        messages = []
        if system.strip():
            messages.append({"role": "system", "content": system.strip()})
        messages.append({"role": "user", "content": user.strip()})

        resp = self.model.invoke(messages)
        return resp.content if hasattr(resp, "content") else str(resp)


LLM = Gemini()
print(LLM.complete(system="", user="Hello"))


Hello there! How can I help you today?



In [8]:
@dataclass
class Constraint:
    tech_stack : Optional[str] = None
    deployment : Optional[str]= None
    priority : Optional[Literal["high", "medium", "low"]] = None 

In [9]:
@dataclass
class ProjectSpec:
    project_name : str
    description : str
    features : List[str]
    constraints : Constraint

In [10]:
class OrchestratorState(TypedDict, total = False):
    spec = Dict[str, Any]

    #Artifacts 
    user_stories: List[Dict[str, Any]]
    architecture : Dict[str, Any]
    plan : List[Dict[str, Any]]
    code_changes : Dict[str, str]
    tests : Dict[str, str]
    test_report : Dict[str, str]
    ci_cd : Dict[str, str]
    deployment : Dict[str, str]

    #these are for logging each nde in langgraph
    approvals : Dict[str, Any]
    repo_url : Optional[str]
    run_id : Optional[str]

In [12]:
class HitlProvider:
    def request(self, gate: str, payload: Dict[str, Any]) -> Dict[str, Any]:
        raise NotImplementedError


class CLIHitl(HitlProvider):
    def request(self, gate: str, payload: Dict[str, Any]) -> Dict[str, Any]:
        print("\n===== HITL GATE:", gate, "=====")

        print(json.dumps(payload, indent=2)[:4000])

        while True:
            ans = input("Approve? [y/n/e=edit]: ").strip().lower()
            if ans in {"y", "n", "e"}:
                break

        notes = ""

        if ans == "e":
            print("Enter revised JSON (end with an empty line):")
            buf = []
            while True:
                line = input()
                if not line:
                    break
                buf.append(line)
            try:
                revised = json.loads("\n".join(buf))
                payload = revised
            except Exception as e:
                print("Invalid JSON, keeping original:", e)
        
        elif ans == "n":
            notes = input("Reason for rejection? ")
        
        return {"approved": ans == "y", "payload": payload, "notes": notes, "by": "cli-user"}

In [13]:
hitl_provider : HitlProvider = CLIHitl()

In [17]:
def normalize_spec(raw : Dict[str, Any]) -> ProjectSpec:
    constraints = raw.get("constraints") or {}
    return ProjectSpec(
        project_name = raw.get("project_name", "Untitled Project"),
        description = raw.get("description", ""),
        features = raw.get("features", []),
        constraints= Constraint(
            tech_stack = constraints.get("tech_stack"),
            deployment = constraints.get("deployment"),
            priority = constraints.get("priority")
        ),
    )

In [18]:
def to_json(data : Any) -> str:
    return json.dumps(data, indent=2, ensure_ascii=False)

In [30]:
import re
def extract_json(text: str):
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except json.JSONDecodeError:
            return {}
    return {}

In [35]:
# def requirements_node(state : OrchestratorState) -> OrchestratorState:
#     spec = normalize_spec(state["spec"])
#     prompt_sys = (
#         "You are a senior product manager. Convert the input spec into precise, testable user stories. "
#         "Use INVEST. Include acceptance criteria and non-functional requirements when relevant."
#     )

#     prompt_user = f"SPEC:\n{to_json(asdict(spec))}\nReturn json with fields: user_stories[]."
    
#     text = LLM.complete(system=prompt_sys, user=prompt_user)
#     print("LLM response:", text[:4000])

#     try:
#         data = json.loads(text)
#         stories = data.get("user_stories", [])
#     except Exception:
#         stories = []
#     return {**state, "user_stories": stories}
    
def requirements_node(state: OrchestratorState) -> OrchestratorState:
    spec = normalize_spec(state["spec"])
    prompt_sys = (
        "You are a senior product manager. Convert the input spec into precise, testable user stories. "
        "Use INVEST. Include acceptance criteria and non-functional requirements when relevant."
    )
    prompt_user = f"SPEC:\n{to_json(asdict(spec))}\nReturn JSON with fields: user_stories[]."

    text = LLM.complete(system=prompt_sys, user=prompt_user)
    # print("LLM response (truncated):", text[:1000])

    # Extract the first valid JSON object
    data = extract_json(text)
    stories = data.get("user_stories", [])
    return {**state, "user_stories": stories}


In [36]:
if __name__ == "__main__":
    sample_state = {
        "spec": {
            "project_name": "Task Manager",
            "description": "A simple task management app",
            "features": ["add task", "list tasks", "mark task done"],
            "constraints": {"tech_stack": "Python + FastAPI"}
        }
    }

    result = requirements_node(sample_state)
    print(json.dumps(result, indent=2))

{
  "spec": {
    "project_name": "Task Manager",
    "description": "A simple task management app",
    "features": [
      "add task",
      "list tasks",
      "mark task done"
    ],
    "constraints": {
      "tech_stack": "Python + FastAPI"
    }
  },
  "user_stories": [
    {
      "id": "US001",
      "title": "As a user, I want to add a new task to the task manager so that I can keep track of my to-dos.",
      "description": "Allows users to add tasks with a description and optional due date.",
      "acceptance_criteria": [
        "The system shall allow users to input a task description (text field, minimum 1 character).",
        "The system shall allow users to optionally input a due date (date picker).",
        "The system shall display a success message upon successful task creation.",
        "The system shall handle invalid input (e.g., empty description) gracefully with appropriate error messages.",
        "The newly added task should be immediately visible in the